In [1]:
!pip install flaml

In [2]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.model_selection import StratifiedKFold, cross_val_score

from sklearn.metrics import accuracy_score

from flaml import AutoML

SEED = 42 # Muito importante manter a SEED igual em todos os modelos para garantir a consistência dos dados no ensemble
FOLDS = 5 # Muito importante manter o mesmo número de FOLDS em todos os modelos para garantir a consistência dos dados no ensemble

/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sintetico = pd.read_csv('../../src/train/train.csv', index_col='id')
original = pd.read_csv('../../src/train/original.csv')
test = pd.read_csv('../../src/test/test.csv', index_col='id')

train = pd.concat([sintetico, original], ignore_index=True)

initial_features = list(test.columns)

In [4]:
def cross_validation(model, X, y, encoder, scoring=accuracy_score):

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

    scores = []
    out_of_fold = []
    
    for i, (train_index, val_index) in enumerate(skf.split(X, y)):
        print(f"Fold {i + 1}")
        
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]

        X_val = X.iloc[val_index]
        y_val = y.iloc[val_index]
        
        
        model.fit(X_train, y_train, task="classification",metric='roc_auc_ovo',time_budget=3600*3)

        probabilidades = model.predict_proba(X_val)

        # Recuperar a predição final a partir das probabilidades
        indices_predicoes = np.argmax(probabilidades, axis=1)
        classes_preditas = model.classes_[indices_predicoes]

        score = scoring(y_val, classes_preditas)

        scores.append(score)

        true_label = pd.Series(y_val.values, name='true')

        pred_label_df = pd.DataFrame(probabilidades)

        oof_pred = pd.concat([pred_label_df, true_label], axis=1, ignore_index=True)
        oof_pred.columns = [f'pred_{encoder[model.classes_[0]]}', f'pred_{encoder[model.classes_[1]]}', f'pred_{encoder[model.classes_[2]]}', 'true']

        out_of_fold.append(oof_pred)

    print(f"Score: {np.mean(scores)}")
    
    return scores, out_of_fold

In [5]:
def save_oof(oof):

    os.makedirs('oof', exist_ok=True)

    for i, fold in enumerate(oof):
        fold.to_csv(f'oof/fold_{i+1}.csv', index=False)

In [6]:
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
train.head()

,Maritalstatus,Applicationmode,Applicationorder,Course,Daytimeeveningattendance,Previousqualification,Previousqualificationgrade,Nacionality,Mothersqualification,Fathersqualification,...,Curricularunits2ndsemcredited,Curricularunits2ndsemenrolled,Curricularunits2ndsemevaluations,Curricularunits2ndsemapproved,Curricularunits2ndsemgrade,Curricularunits2ndsemwithoutevaluations,Unemploymentrate,Inflationrate,GDP,Target
0,1,1,1,9238,1,1,126.0,1,1,19,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,1,1,3,9500,1,1,131.0,1,19,3,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [7]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('object')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [8]:
train = reduce_mem_usage(train)
train.info()

Memory usage of dataframe is 22.85 MB
Memory usage after optimization is: 4.17 MB
Decreased by 81.8%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80942 entries, 0 to 80941
Data columns (total 37 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Maritalstatus                            80942 non-null  int8   
 1   Applicationmode                          80942 non-null  int8   
 2   Applicationorder                         80942 non-null  int8   
 3   Course                                   80942 non-null  int16  
 4   Daytimeeveningattendance                 80942 non-null  int8   
 5   Previousqualification                    80942 non-null  int8   
 6   Previousqualificationgrade               80942 non-null  float16
 7   Nacionality                              80942 non-null  int8   
 8   Mothersqualification                     80942 non-null  int8   
 9   Fathersqualific

In [9]:
y = train.pop('Target')
X = train

encoder = {
        'Graduate':'Graduate',
        'Enrolled':'Enrolled',
        'Dropout':'Dropout'
           }

In [10]:
automl = AutoML()

In [11]:
scores,oof = cross_validation(automl, X, y, encoder)

Fold 1
[flaml.automl.logger: 06-10 23:13:38] {1680} INFO - task = classification
[flaml.automl.logger: 06-10 23:13:38] {1691} INFO - Evaluation method: cv
[flaml.automl.logger: 06-10 23:13:38] {1789} INFO - Minimizing error metric: 1-roc_auc_ovo
[flaml.automl.logger: 06-10 23:13:38] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 06-10 23:13:38] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 06-10 23:13:40] {2345} INFO - Estimated sufficient time budget=20259s. Estimated necessary time budget=497s.
[flaml.automl.logger: 06-10 23:13:40] {2392} INFO -  at 2.5s,	estimator lgbm's best error=0.1166,	best estimator lgbm's best error=0.1166
[flaml.automl.logger: 06-10 23:13:40] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 06-10 23:13:42] {2392} INFO -  at 4.4s,	estimator lgbm's best error=0.1166,	best estimator lgbm's best error=0.1166
[flaml.aut

/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.automl.logger: 06-10 23:44:53] {2392} INFO -  at 1875.7s,	estimator lrl1's best error=0.1379,	best estimator xgb_limitdepth's best error=0.0718
[flaml.automl.logger: 06-10 23:44:53] {2219} INFO - iteration 146, current learner lrl1


/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.w

[flaml.automl.logger: 06-10 23:45:22] {2392} INFO -  at 1904.7s,	estimator lrl1's best error=0.1379,	best estimator xgb_limitdepth's best error=0.0718
[flaml.automl.logger: 06-10 23:45:22] {2219} INFO - iteration 147, current learner xgboost


...

[flaml.automl.logger: 06-11 11:26:04] {2392} INFO -  at 585.6s,	estimator extra_tree's best error=0.0826,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:26:04] {2219} INFO - iteration 105, current learner lgbm
[flaml.automl.logger: 06-11 11:26:20] {2392} INFO -  at 602.0s,	estimator lgbm's best error=0.0729,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:26:20] {2219} INFO - iteration 106, current learner extra_tree
[flaml.automl.logger: 06-11 11:26:23] {2392} INFO -  at 604.4s,	estimator extra_tree's best error=0.0826,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:26:23] {2219} INFO - iteration 107, current learner rf
[flaml.automl.logger: 06-11 11:26:26] {2392} INFO -  at 607.5s,	estimator rf's best error=0.0813,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:26:26] {2219} INFO - iteration 108, current learner extra_tree
[flaml.automl.logger: 06-11 11:26:28] {2392} INFO -  at 

/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.automl.logger: 06-11 11:26:57] {2392} INFO -  at 638.4s,	estimator lrl1's best error=0.1373,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:26:57] {2219} INFO - iteration 110, current learner rf


/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.automl.logger: 06-11 11:26:59] {2392} INFO -  at 640.5s,	estimator rf's best error=0.0813,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:26:59] {2219} INFO - iteration 111, current learner xgb_limitdepth
[flaml.automl.logger: 06-11 11:26:59] {2392} INFO -  at 641.2s,	estimator xgb_limitdepth's best error=0.0772,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:26:59] {2219} INFO - iteration 112, current learner extra_tree
[flaml.automl.logger: 06-11 11:27:03] {2392} INFO -  at 644.4s,	estimator extra_tree's best error=0.0826,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:27:03] {2219} INFO - iteration 113, current learner lgbm
[flaml.automl.logger: 06-11 11:27:27] {2392} INFO -  at 668.9s,	estimator lgbm's best error=0.0729,	best estimator xgboost's best error=0.0725
[flaml.automl.logger: 06-11 11:27:27] {2219} INFO - iteration 114, current learner xgb_limitdepth
[flaml.automl.logger: 06-11 11:27:29

In [16]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best roc_auc_ovo  on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgb_limitdepth
Best hyperparmeter config: {'n_estimators': 7407, 'max_depth': 5, 'min_child_weight': 25.42430935717195, 'learning_rate': 0.008714483398941934, 'subsample': 1.0, 'colsample_bylevel': 0.7009027524282787, 'colsample_bytree': 0.45758755737098006, 'reg_alpha': 0.0011539001566573503, 'reg_lambda': 49.616612455809346}
Best roc_auc_ovo  on validation data: 0.9286
Training duration of best run: 57.45 s


In [13]:
for fold in oof:
    fold['true'] = fold['true'].replace(encoder)

save_oof(oof)